Crée un fichier csv qui va contenir, pour chaque ville, les informations météo + hôtels

In [2]:
import pandas as pd
from variables import cities_csv_name, hotels_jl_name, weather_csv_name, final_csv_name

# Villes = 1er dataframe
df_cities = pd.read_csv(cities_csv_name)
df_cities = df_cities.set_index("id")

# Météo = 2ème dataframe
df_weather = pd.read_csv(weather_csv_name)

# On fusionne les 2 dataframe "Villes" et "Météo" en un seul
df_cities_weather = df_cities.merge(df_weather, how="outer", left_on="id", right_on="id_city")
df_cities_weather = df_cities_weather.set_index("id_city")

# Création d'un dataframe qui, pour chaque ville, donne :
# - les températures / températures ressenties / vitesse du vent / proba précipitations moyens sur les 5 prochains jours
# - la condition météo (couvert, pluvieux, ...) la plus donnée pour les 5 prochains jours
# Ce dataframe va nous permettre de construire le csv final + construire la carte météo
df_weather_5_days = df_cities_weather.groupby(["id_city","city", "lat", "lon"]).agg({"temperature" : "mean", 
                                                                                        "temperature_feels_like" : "mean",
                                                                                        "condition" : lambda x: x.mode()[0],
                                                                                        "wind_speed" : "mean",
                                                                                        "proba_precipitation" : "mean"}).reset_index()
df_weather_5_days["id_city"] = df_weather_5_days["id_city"].astype(int)

# Hotels = 3ème DataFrame
df_hotels = pd.read_json(hotels_jl_name, lines = True)

# Création d'un dataframe final qui combine les infos météo pour les 5 prochains jours ainsi que le top 20 des hôtels
df_final = df_weather_5_days.merge(df_hotels, on=["id_city", "city"])
df_final = df_final.set_index("id_city")
df_final.to_csv(final_csv_name)